In [1]:
import torch
import torchvision 
import pandas as pd
import numpy as np
import argparse
from PIL import Image
import os
from tqdm import tqdm
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from torch.utils import data
from torchvision import transforms, datasets, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
import time
import utils_ObjectDetection as utils
from torch.utils.data import BatchSampler,SequentialSampler,RandomSampler
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import math 
from itertools import combinations
import ITB_RECALL_PRECISION_F1 as rp_f1

In [2]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

def collate_fn(batch):
    return tuple(zip(*batch))


def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)


def get_model_instance_segmentation(num_classes,train_layer):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True,trainable_backbone_layers=train_layer)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def plot_image_from_output(img, annotation):
    
    img = img.cpu().permute(1,2,0)
    
    fig,ax = plt.subplots(figsize=(20, 10))
    ax.imshow(img)
    
    for idx in range(len(annotation["boxes"])):
        xmin, ymin, xmax, ymax = annotation["boxes"][idx]

        if annotation['labels'][idx] == 1 :
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='b',facecolor='none')
        
        elif annotation['labels'][idx] == 2 :
            
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            
        elif annotation['labels'][idx] == 3 :
        
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='orange',facecolor='none')
        else:
            rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        ax.add_patch(rect)

    return plt


In [3]:

class simpleDataset(object):


    def __init__(self, dataset,resize,color, img_foler_path,transforms=None):
    #             self.root = root
            self.transforms = transforms
            self.adnoc = dataset
            self.ids = dataset.index
            self. filenames=dataset['path'].to_list()
            self.resize =resize
            self.color=color
            self.img_foler_path=img_foler_path
            
    def __getitem__(self, index):
        # Own coco file
        adnoc_df = self.adnoc
        # Image ID ,폴더에서 index번째 
        img_id = self.ids[index]

       # List: get annotation id from coco, index번째 annotation가져오기 [[1,2,3,4],[5,6,7,8]]
        annotation = adnoc_df['box'][img_id]


        # open the input image, 흑백=L , 단색=1
        img= Image.open(str(self.img_foler_path)+str(adnoc_df.loc[img_id]['path'])).convert(self.color)
        img= img.resize((int(img.width / self.resize), int(img.height / self.resize)))
      
        # number of objects in the image
        num_objs = len(annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        areas = []
        label = []
        for i in range(num_objs):

            xmin = annotation[i][0]
            ymin = annotation[i][1]
            xmax = xmin + annotation[i][2]
            ymax = ymin + annotation[i][3]
            l=annotation[i][4]
            area=annotation[i][2]*annotation[i][3]

            boxes.append([xmin/self.resize, ymin/self.resize, xmax/self.resize, ymax/self.resize])
          
            label.append(l)
            areas.append(area)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        #areas= box size = width * height
        areas = torch.as_tensor(areas, dtype=torch.float32)

        # Labels (In my case, I only one class: target class or background)
        labels = torch.as_tensor(label, dtype=torch.int64)

        # Tensorise img_id
        img_id = torch.tensor([img_id])

        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)



    # the total number of samples (optional)
    def __len__(self):
        return len(self.filenames)

    


In [18]:
def result_RPA(test_data_loader,model,param,iteration):
    
    model = get_model_instance_segmentation(4,5)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    model.load_state_dict(torch.load(param))
    model.eval()


    confi=[0.5]
    RECALL=[]
    PRECISION=[]
    F1_SCORE=[]
    MEAN_R=[]
    MEAN_P=[]
    MEAN_F1=[]
    
    for c in confi: 
        labels = []
        preds_adj_all = []
        annot_all = []
        print('confidence=',c)
        for im, annot in tqdm(test_data_loader, position = 0, leave = True):
            im = list(img.to(device) for img in im)

   
            for t in annot:
                labels += t['labels']

            with torch.no_grad():
                preds_adj = make_prediction(model, im, c)
                preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
                preds_adj_all.append(preds_adj)
                annot_all.append(annot)

        #precision=0.2
        iou_threds=0.5
        pr_f1=rp_f1.Precision_Recall_F1(iou_threds,preds_adj_all,annot_all)
      
        
        recall,mean_r=pr_f1.RECALL()
        precision,mean_p=pr_f1.PRECISION()
        f1_score,mean_f1=pr_f1.F1_SCORE()
      

        RECALL.append(recall)
        PRECISION.append(precision)
        F1_SCORE.append(f1_score)
        MEAN_R.append(mean_r)
        MEAN_P.append(mean_p)
        MEAN_F1.append(mean_f1)
        
    print(MEAN_R,'\n',MEAN_P,'\n',MEAN_F1)
    result=pd.DataFrame(confi,columns=['confidence'])
    result['iteration']=iteration
    result['recall']=[RECALL]
    result['mean_recall']=MEAN_R
    result['precision']=[PRECISION]
    result['mean_precision']=MEAN_P
    result['f1_form']=[F1_SCORE]
    result['mean_f1']=MEAN_F1
      
    
    return result

In [5]:
def random_sampling(train,batch):
   
    sampled=train.sample(n=batch,random_state=42)
    train_rest=train.drop(sampled.index)
        
    sample_dataset = simpleDataset(dataset=sampled,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/itb_p_500/',
                                  transforms=get_transform())
    
    sample_data_loader = torch.utils.data.DataLoader(sample_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)
    return sampled,train_rest,sample_data_loader


In [6]:
def data_load():
    
    file_path='D:/OBJ_PAPER/'
    itb=pd.read_pickle(file_path+'itb.pkl')

    test=itb.sample(frac=0.2,random_state=42)
    train_valid=itb.drop(test.index)
    valid=train_valid.sample(n=50, random_state=42)
    train=train_valid.drop(valid.index)


    test.reset_index(drop=True,inplace=True)
    test['index']=test.index

    valid.reset_index(drop=True,inplace=True)
    valid['index']=valid.index

    train.reset_index(drop=True,inplace=True)
    train['index']=train.index
    print(' train shape : ',train.shape, 'valid shape : ', valid.shape, 'test shape : ', test.shape, )

    # create own Dataset
    train_dataset = simpleDataset(dataset=train,
                                  resize=4,
                                  color='L',
                                  img_foler_path=file_path+'Data/itb_p_500/',
                                  transforms=get_transform())

    valid_dataset = simpleDataset(
                              dataset=valid,
                              resize=4,
                              color='L',
                              img_foler_path=file_path+'Data/itb_p_500/',
                              transforms=get_transform())

    test_dataset = simpleDataset(
                              dataset=test,
                              resize=4,
                              color='L',
                              img_foler_path=file_path+'Data/itb_p_500/',
                              transforms=get_transform())



    train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                              batch_size=1,
                                              shuffle=False,
                                              num_workers=0,                                         
                                              collate_fn=collate_fn)

    valid_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)

    test_data_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=0,
                                          collate_fn=collate_fn)

    return test,valid,train,train_data_loader,valid_data_loader, test_data_loader


In [7]:
def data_load(data):
 
    # create own Dataset
    dataset = simpleDataset(dataset=data,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                                  transforms=get_transform())



    data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=0,
                                          collate_fn=collate_fn)

    return data_loader

# Uncertianty samplig

### 불확실성이 큰것 top-k 개 샘플링

In [8]:
class uncertainty_sampling:
    '''
    'uncertainty', : a
     'class_difficulty', : b
     'class_ambiguity',: c
        'class_sparse', x
     'al_score'
    '''
    def __init__(self,model,confidence,batch,train_data_loader,method,param):
        
        
        self.model=model
        self.confidence=confidence
        self.batch=batch
        self.train_data_loader=train_data_loader
        self.method=method
        self.a=param[0]
        self.b=param[1]
        self.c=param[2]
        
    def bbox_iou(self,box1, box2, x1y1x2y2=True):
        """
        Returns the IoU of two bounding boxes
        """
        box1=torch.tensor(box1)
        box2=torch.tensor(box2)

        if not x1y1x2y2:
            # Transform from center and width to exact coordinates
            b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
            b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
            b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
            b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
        else:
            # Get the coordinates of bounding boxes
            b1_x1, b1_y1, b1_x2, b1_y2 = box1[0], box1[1], box1[2], box1[3]
            b2_x1, b2_y1, b2_x2, b2_y2 = box2[0], box2[1], box2[2], box2[3]

        # get the corrdinates of the intersection rectangle
        inter_rect_x1 = torch.max(b1_x1, b2_x1)
        inter_rect_y1 = torch.max(b1_y1, b2_y1)
        inter_rect_x2 = torch.min(b1_x2, b2_x2)
        inter_rect_y2 = torch.min(b1_y2, b2_y2)
        # Intersection area
        inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
        # Union Area
        b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
        b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

        iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

        return iou

    def img_uncertainty(self,model,ascend):
        '''
        내림차순, 큰 수 부터
        ascend=False
        오름차순, 작은 수 부터
        ascend=True 
        
        '''
        img_id=[]
        pred_label=[]
        pred_scores=[]
        labels = []
        bbox=[]
        img_uncertianty=[]
        
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        model.to(device)
        
        for im, annot in tqdm(self.train_data_loader, position = 0, leave = True):
            im = list(img.to(device) for img in im)
            #test data 이미지 id 저장
            img_id.append(annot[0]['image_id'].item())

            for t in annot:
                labels += t['labels']

            with torch.no_grad():
                preds_adj = make_prediction(model, im, self.confidence)
                preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]

                #예측 label 저장
                pred_label.append(preds_adj[0]['labels'].tolist())
                #box
                bbox.append((preds_adj[0]['boxes'].tolist()))
                #예측 score 저장
                pred_scores.append(preds_adj[0]['scores'].tolist())
                img_uncertianty.append(np.mean(preds_adj[0]['scores'].tolist()))

        model_pred=pd.DataFrame(img_id,columns=['img_id'])
        model_pred['label']=pred_label
        model_pred['score']=pred_scores
        model_pred['bbox']=bbox
        #confidence가 0.5이상인 객체의 score 합
        uncertainty=[]
        for i in range(len(model_pred)):

            scores=model_pred.iloc[i]['score']
            un=0
            for s in scores:

                if s>self.a:
                    un=un+s
            uncertainty.append(un)

        model_pred['uncertainty']=uncertainty
        
        #confidence가 0.3이하인 객체의 개수
        class_difficulty=[]
        for i in range(len(model_pred)):
            cls_dif=0
            scores=model_pred.iloc[i]['score']
            for s in scores:
                if s<self.b:
                    cls_dif+=1
            class_difficulty.append(cls_dif)

        model_pred['class_difficulty']=class_difficulty
        
        #iou 계산하여 클래스가 다르면서 중첩인 box개수
        class_ambiguity=[]
        for a in range(len(model_pred)):
            box_index=list(range(len(model_pred.iloc[a]['bbox'])))
            label_index=list(range(len(model_pred.iloc[a]['label'])))
            iou_index=list(combinations(box_index, 2))

            cls_amb=0
            for i in iou_index:
                box1_i=i[0]
                box2_i=i[1]

                #box좌표
                box1=model_pred.iloc[a]['bbox'][box1_i]
                box2=model_pred.iloc[a]['bbox'][box2_i]

                #라벨
                box1_label=model_pred.iloc[a]['label'][box1_i]
                box2_label=model_pred.iloc[a]['label'][box2_i]

                #class가 다른 경우

                if (box1_label!= box2_label):
                    #iou 계산
                    iou=self.bbox_iou(box1, box2, x1y1x2y2=True)
                    if iou>self.c:
                        cls_amb+=1
            class_ambiguity.append(cls_amb)        
                #클래스가 같은경우

        model_pred['class_ambiguity']=class_ambiguity
        
        #class sparse, 예측된 라벨 중 특정 라벨의 수가 적은 경우 해당 라벨을 우선적으로 선택
        text=0
        form=0
        table=0

        for i in range(len(model_pred)):

            label_line=model_pred['label'].iloc[i]
            for l in label_line:
                if l==1:
                    form+=1

                elif l==2:
                    table+=1

                else:
                    text+=1


        total=text+form+table
        text_ratio=round(text/total,5)
        form_ratio=round(form/total,5)
        table_ratio=round(table/total,5)
        tt=[text_ratio,form_ratio,table_ratio]
        total_ratio=pd.DataFrame(tt,columns=['label'])
        total_ratio.index=['text_spars','form_spars','table_spars']
        #비율이 가장 낮은 label
        spars=total_ratio.sort_values(by=['label'],ascending=True).index[0]



        text_spars=[]
        form_spars=[]
        table_spars=[]

        for i in range(len(model_pred)):

            img_text=0
            img_form=0
            img_table=0

            label_line=model_pred['label'].iloc[i]
            for l in label_line:
                if l==1:
                    img_form+=1
                elif l==2:
                    img_table+=1
                else:
                    img_text+=1

            text_spars.append(img_text*text_ratio)
            form_spars.append(img_form*form_ratio)
            table_spars.append(img_table*table_ratio)

        model_pred['text_spars']=text_spars
        model_pred['form_spars']=form_spars
        model_pred['table_spars']=table_spars
        
        model_pred['class_sparse']=model_pred[spars]
        model_pred['al_score']=model_pred[spars]+model_pred['uncertainty']+model_pred['class_difficulty']+model_pred['class_ambiguity']
        model_pred.sort_values(by=[self.method],ascending=False,inplace=True)
             
        
        return model_pred




    def sampling(self,train,ascend):
        

        model_pred=self.img_uncertainty(self.model,ascend)
        sample_idx=model_pred[:self.batch]['img_id'].tolist()
        sample=train.loc[sample_idx]
        train_rest=train.drop(sample.index)
        train_rest_dataset = simpleDataset(dataset=train_rest,
                              resize=4,
                              color='L',
                              img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                              transforms=get_transform())

        train_rest_data_loader = torch.utils.data.DataLoader(train_rest_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)

        sample_dataset = simpleDataset(dataset=sample,
                                  resize=4,
                                  color='L',
                                  img_foler_path='D:/OBJ_PAPER/Data/itb_p_500/',
                                  transforms=get_transform())

        sample_data_loader = torch.utils.data.DataLoader(sample_dataset,
                                              batch_size=1,
                                              shuffle=True,
                                              num_workers=0,
                                              collate_fn=collate_fn)

      
        
        return model_pred,sample,train_rest,train_rest_data_loader

In [201]:
Total=[100,200,300,343]
Batch=30
train=train1
valid=valid1
test=test
Train_param_path='D:/OBJ_PAPER/model_param/FRCN_itb_random1_batch30_total{}_param.pt'
RESULT_path='D:/OBJ_PAPER/result/FRCN_random1_b30_total_{}_validation_p{}.csv'
Sampled_img_path='D:/OBJ_PAPER/result/FRCN_random1_b30_total_{}_validation_p{}_sampled.csv'

TL_train_Random(Total,Batch,train,valid,test,Train_param_path,RESULT_path,Sampled_img_path)

# Active Learning

In [15]:
data_path='D:/OBJ_PAPER/Data/3_fold_cv/'

train1=pd.read_pickle(data_path+'train1.pkl')
train2=pd.read_pickle(data_path+'train2.pkl')
train3=pd.read_pickle(data_path+'train3.pkl')

valid1=pd.read_pickle(data_path+'valid1.pkl')
valid2=pd.read_pickle(data_path+'valid2.pkl')
valid3=pd.read_pickle(data_path+'valid3.pkl')

test=pd.read_pickle(data_path+'test.pkl')

In [19]:
def train_AL(train,valid,test,total,method,method_param,param_name,TL_param,Train_param_path,result_path,sample_path):  
       
    num_classes = 4
    num_epochs = 100
    train_layer=5
    batch=30
    patience=10
    ascend=False
    
    model = get_model_instance_segmentation(num_classes,train_layer)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    # move model to the right device
    model.to(device)
    model.load_state_dict(torch.load(TL_param))

    # parameters
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.0001)
    
    iteration = 0    

    RESULT=pd.DataFrame()
    Sampled_img=pd.DataFrame()
    
    #모델 저장 
    train_param_path=Train_param_path.format(total,method,param_name)
    
    #data reload
   
    train_data_loader=data_load(train)
    valid_data_loader=data_load(valid)
    test_data_loader=data_load(test)
    
    while len(Sampled_img)<total:

        if len(Sampled_img)+batch>total:
            batch=(len(Sampled_img)+batch)-total
            sample=train.sample(n=batch)
            Sampled_img=Sampled_img.append(sample,ignore_index=True)
            sample_data_loader=data_load(Sampled_img)
            
        else:
            us=uncertainty_sampling(model,0.00001,batch,train_data_loader,method,method_param)

            model_pred,sampled,train_rest,train_rest_data_loader=us.sampling(train,ascend)

            Sampled_img=Sampled_img.append(sampled,ignore_index=True)  
            sample_data_loader=data_load(Sampled_img)
            
            #sample 제외한 나머지 train 데이터
            train=train_rest

            #ranodm으로 가져온 N개 데이터
            train_data_loader=train_rest_data_loader


        iteration += 1
        print(iteration)
        model.train()
        not_save_count=0
        
        #평균 loss
        avg_train_loss=[]
        avg_valid_loss=[]
        
        for epoch in range(num_epochs):

            # 모델이 학습되는 동안 trainning loss를 track
            train_losses = []
            # 모델이 학습되는 동안 validation loss를 track
            valid_losses = []
            
            st = time.time()
            for imgs, annotations in sample_data_loader:

                imgs = list(img.to(device) for img in imgs)
                annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
                loss_dict = model(imgs, annotations)
                #batch size=10, 10개 loss 각각 도출
                losses = sum(loss for loss in loss_dict.values())
                train_losses.append(losses.item())       
                optimizer.zero_grad()
                losses.backward()
                optimizer.step()

            #이미지 한장당 평균 loss
            avg_train_loss.append(np.mean(train_losses).round(5))

            #validation, early_stop, save weights
            with torch.no_grad():

                for im, annot in valid_data_loader:
                    im = list(img.to(device) for img in im)
                    annot = [{k: v.to(device) for k, v in t.items()} for t in annot]
                    val_loss_dict = model(im, annot)
                    val_losses = sum(val_loss for val_loss in val_loss_dict.values())
                    valid_losses.append(val_losses.item())

                epoch_val_loss=np.mean(valid_losses).round(5)
                avg_valid_loss.append(epoch_val_loss)  

            fi = time.time()     
            print('epoch:',epoch,'train_loss: ',np.mean(train_losses).round(5),'validation loss : ',np.mean(valid_losses).round(5),'time',fi-st)


            min_val_loss=np.min(avg_valid_loss)
            if min_val_loss>=epoch_val_loss:

                torch.save(model.state_dict(),train_param_path)
                not_save_count=0
                print('epoch:',epoch,'save model')

            else:
                not_save_count+=1
                model.load_state_dict(torch.load(train_param_path))
                if not_save_count>=patience:
                    print('no more training')
                    break


        fi = time.time()     
        print('iteration:',iteration,'train_loss: ',np.mean(train_losses).round(5),'time',fi-st)
        print('sample num:',len(Sampled_img))
        model.load_state_dict(torch.load(train_param_path))

        result_batch=result_RPA(test_data_loader,model,train_param_path,iteration)
        RESULT=RESULT.append(result_batch,ignore_index=True)

    RESULT.to_csv(result_path.format(total,method,param_name),index=False)   
    Sampled_img.to_csv(sample_path.format(total,method,param_name),index=False) 
    return RESULT

In [ ]:
TL_param='D:/OBJ_PAPER/model_param/model_fasterrcnn_bach5_p10_50_param.pt'
Train_param_path='D:/OBJ_PAPER/model_param/FRCN_AL1_batch30_total{}_{}_{}param.pt'
result_path='D:/OBJ_PAPER/result/FRCN_AL1_batch30_total{}_{}_{}_result.csv'
sample_path='D:/OBJ_PAPER/result/FRCN_AL1_batch30_total{}_{}_{}_sampled.csv'


train=train1
valid=valid1
test=test



total=180
Method= [
     'class_difficulty',
     'class_ambiguity',
]

a=[0.7,0.9]
b=[0.1,0.5]
c=[0.7,0.9]

#param=[un_param,diff_param,ambi_param]

for method in Method:
    if method=='uncertainty':
        for a_param in a:
            print('total:',total,'method:',method,'a_param:',a_param)
            method_param=[a_param,0.3,0.5]
            train_AL(train,valid,test,total,method,method_param,a_param,TL_param,Train_param_path,result_path,sample_path)
            
    elif method=='class_difficulty':
        for b_param in b:
            print('total:',total,'method:',method,'b_param:',b_param)
            method_param=[0.5,b_param,0.5]
            train_AL(train,valid,test,total,method,method_param,b_param,TL_param,Train_param_path,result_path,sample_path)
            
    elif method=='class_ambiguity':
        for c_param in c:
            print('total:',total,'method:',method,'c_param:',c_param)
            method_param=[0.5,0.3,c_param]
            train_AL(train,valid,test,total,method,method_param,c_param,TL_param,Train_param_path,result_path,sample_path)         

total: 180 method: class_difficulty b_param: 0.1


100%|████████████████████████████████████████████████████████████████████████████████| 348/348 [00:29<00:00, 11.93it/s]


1
epoch: 0 train_loss:  0.42246 validation loss :  0.22442 time 9.256716966629028
epoch: 0 save model
epoch: 1 train_loss:  0.24724 validation loss :  0.1872 time 9.180119752883911
epoch: 1 save model
epoch: 2 train_loss:  0.17198 validation loss :  0.19799 time 9.214881420135498
epoch: 3 train_loss:  0.16481 validation loss :  0.22946 time 9.20757508277893
epoch: 4 train_loss:  0.16839 validation loss :  0.2043 time 9.246517419815063
epoch: 5 train_loss:  0.16776 validation loss :  0.2378 time 9.23314619064331
epoch: 6 train_loss:  0.16321 validation loss :  0.21004 time 9.228680849075317
epoch: 7 train_loss:  0.16565 validation loss :  0.23407 time 9.34303331375122
epoch: 8 train_loss:  0.16757 validation loss :  0.22818 time 9.23823356628418
epoch: 9 train_loss:  0.17963 validation loss :  0.19368 time 9.273492097854614
epoch: 10 train_loss:  0.17901 validation loss :  0.21327 time 9.253748655319214
epoch: 11 train_loss:  0.17281 validation loss :  0.21638 time 9.25045132637024
no m

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.86it/s]


precision [0.39785, 0.89873, 0.6484]
recall [0.69811, 0.98611, 0.91026]
[0.86483] 
 [0.64833] 
 [0.7410917998096697]


...

2
epoch: 0 train_loss:  0.30633 validation loss :  0.26801 time 14.453686952590942
epoch: 0 save model
epoch: 1 train_loss:  0.2376 validation loss :  0.18075 time 14.472286462783813
epoch: 1 save model
epoch: 2 train_loss:  0.17783 validation loss :  0.17241 time 14.409119129180908
epoch: 2 save model
epoch: 3 train_loss:  0.1516 validation loss :  0.16833 time 14.576417922973633
epoch: 3 save model
epoch: 4 train_loss:  0.13331 validation loss :  0.16354 time 14.550203084945679
epoch: 4 save model
epoch: 5 train_loss:  0.11618 validation loss :  0.16938 time 14.44685697555542
epoch: 6 train_loss:  0.10835 validation loss :  0.18224 time 14.452032327651978
epoch: 7 train_loss:  0.1126 validation loss :  0.18315 time 14.452722072601318
epoch: 8 train_loss:  0.10972 validation loss :  0.20035 time 14.466431617736816
epoch: 9 train_loss:  0.11141 validation loss :  0.17986 time 14.498121738433838
epoch: 10 train_loss:  0.10759 validation loss :  0.20523 time 14.625690460205078
epoch: 11 

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:09<00:00, 11.07it/s]


precision [0.73846, 0.9726, 0.68664]
recall [0.90566, 0.98611, 0.95513]
[0.94897] 
 [0.79923] 
 [0.8676870988445257]


100%|████████████████████████████████████████████████████████████████████████████████| 288/288 [00:25<00:00, 11.42it/s]


3
epoch: 0 train_loss:  0.243 validation loss :  0.28938 time 19.548088312149048
epoch: 0 save model
